In [1]:
! pip install -m  requirements.txt
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets download ayushmi77al/weather-data-set-for-beginners


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
mkdir: cannot create directory ‘/home/tom/.kaggle’: File exists
Dataset URL: https://www.kaggle.com/datasets/ayushmi77al/weather-data-set-for-beginners
License(s): unknown
100%|█████████████████████████████████████████| 101k/101k [00:00<00:00, 360kB/s]
100%|█████████████████████████████████████████| 101k/101k [00:00<00:00, 359kB/s]


In [2]:
!unzip -o ./weather-data-set-for-beginners.zip 

Archive:  ./weather-data-set-for-beginners.zip
  inflating: 1. Weather Data.csv     


In [3]:
!rm ./weather-data-set-for-beginners.zip

In [4]:
# load data file
import pandas as pd
import numpy as np

data = pd.read_csv("1. Weather Data.csv")

data.head()

,Date/Time,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa,Weather
0,1/1/2012 0:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,1/1/2012 1:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,1/1/2012 2:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"
3,1/1/2012 3:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog"
4,1/1/2012 4:00,-1.5,-3.3,88,7,4.8,101.23,Fog


In [5]:
import sqlite3

In [6]:
# Data cleanup
data.columns = data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

data.head()

,date/time,temp_c,dew_point_temp_c,rel_hum_%,wind_speed_km/h,visibility_km,press_kpa,weather
0,1/1/2012 0:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,1/1/2012 1:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,1/1/2012 2:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"
3,1/1/2012 3:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog"
4,1/1/2012 4:00,-1.5,-3.3,88,7,4.8,101.23,Fog


In [7]:
# Convert CSV to SQLite table
# connect to database
conn = sqlite3.connect('weather.db')

# write data to SQLite database
data.to_sql('weather', conn, if_exists='replace', index=False)

# close connection
conn.close()

In [8]:
from sqlalchemy import create_engine, inspect, text
from sqlalchemy.orm import sessionmaker
import tabulate

engine = create_engine('sqlite:///weather.db')

Session = sessionmaker(bind=engine)
session = Session()


In [9]:
# get table name(s)
insp = inspect(engine)
table = insp.get_table_names()[0]

# get table schema
inspector = inspect(engine)
columns = inspector.get_columns(table)
column_names = [column['name'] for column in columns]
column_names


['date/time',
 'temp_c',
 'dew_point_temp_c',
 'rel_hum_%',
 'wind_speed_km/h',
 'visibility_km',
 'press_kpa',
 'weather']

In [10]:
# query data function

def query_data(query):
    '''
    Executes query on database

    Parameters
    ----------
    query : sqlalchemy.text
        SQL query to be executed
    '''
    with engine.connect() as conn:
        result = conn.execute(query)

    return result

def print_table(query, limit=0 ):
    '''
    Prints query results in tabular format.

    Parameters
    ----------
    query : sqlalchemy.text
        query to be executed
    limit : int
        number of rows to print (default is 0, which prints all rows)
    '''
    data = list(query)
    if len(data) == 0:
        print('No data found')
        return
    
    data = data if not limit else data[:limit]
    print(tabulate.tabulate(data, headers=column_names, tablefmt='psql'))


In [11]:
# Find all records where the weather was exactly clear.

query = text("SELECT * FROM weather WHERE weather = 'Clear'")
print_table(query_data(query), 10)


+----------------+----------+--------------------+-------------+-------------------+-----------------+-------------+-----------+
| date/time      |   temp_c |   dew_point_temp_c |   rel_hum_% |   wind_speed_km/h |   visibility_km |   press_kpa | weather   |
|----------------+----------+--------------------+-------------+-------------------+-----------------+-------------+-----------|
| 1/3/2012 19:00 |    -16.9 |              -24.8 |          50 |                24 |            25   |      101.74 | Clear     |
| 1/5/2012 18:00 |     -7.1 |              -14.4 |          56 |                11 |            25   |      100.71 | Clear     |
| 1/5/2012 19:00 |     -9.2 |              -15.4 |          61 |                 7 |            25   |      100.8  | Clear     |
| 1/5/2012 20:00 |     -9.8 |              -15.7 |          62 |                 9 |            25   |      100.83 | Clear     |
| 1/5/2012 21:00 |     -9   |              -14.8 |          63 |                13 |            2

In [12]:
# Find the number of times the wind speed was exactly 4 km/hr.

query = text('SELECT * FROM weather WHERE "wind_speed_km/h" = 4')
print_table(query_data(query), 10)


+----------------+----------+--------------------+-------------+-------------------+-----------------+-------------+---------------+
| date/time      |   temp_c |   dew_point_temp_c |   rel_hum_% |   wind_speed_km/h |   visibility_km |   press_kpa | weather       |
|----------------+----------+--------------------+-------------+-------------------+-----------------+-------------+---------------|
| 1/1/2012 0:00  |     -1.8 |               -3.9 |          86 |                 4 |             8   |      101.24 | Fog           |
| 1/1/2012 1:00  |     -1.8 |               -3.7 |          87 |                 4 |             8   |      101.24 | Fog           |
| 1/5/2012 0:00  |     -8.8 |              -11.7 |          79 |                 4 |             9.7 |      100.32 | Snow          |
| 1/5/2012 5:00  |     -7   |               -9.5 |          82 |                 4 |             4   |      100.19 | Snow          |
| 1/7/2012 2:00  |     -8.1 |              -11.1 |          79 |     

In [13]:
# Check if there are any NULL values present in the dataset.

for column in column_names:
    query = text(f'SELECT * FROM weather WHERE "{column}" IS NULL')

    if len(list(query_data(query))) > 0:
        print_table(query_data(query), 10)
    else:
        print(f'No NULL values in {column}')


No NULL values in date/time
No NULL values in temp_c
No NULL values in dew_point_temp_c
No NULL values in rel_hum_%
No NULL values in wind_speed_km/h
No NULL values in visibility_km
No NULL values in press_kpa
No NULL values in weather


In [14]:
# Rename the column "Weather" to "Weather_Condition."

exec = text('ALTER TABLE weather RENAME COLUMN weather TO weather_condition')
with engine.connect() as conn:
        result = conn.execute(exec)


# Verify that the column was renamed successfully.

query = text('SELECT * FROM weather LIMIT 1')
with engine.connect() as conn:
    result = conn.execute(query)

# update column names
print(column_names)
column_names = list(result.keys())
print(column_names)



['date/time', 'temp_c', 'dew_point_temp_c', 'rel_hum_%', 'wind_speed_km/h', 'visibility_km', 'press_kpa', 'weather']
['date/time', 'temp_c', 'dew_point_temp_c', 'rel_hum_%', 'wind_speed_km/h', 'visibility_km', 'press_kpa', 'weather_condition']
